# 8. 类型提示

编写类型提示时建议遵守以下代码风格
- 参数名称和 `:` 之间不留空格，`:` 后加一个空格
- 参数默认值前面的 `=` 两侧加空格
- 根据 PEP8，除参数默认值前面的 `=` 之外，其它 `=` 两侧不加空格

可以使用 flake8 报告代码中的问题（但是我选ruff），

使用 blue 来进行格式化代码，black虽然可以格式化，但是不如 blue 遵守 Python 自身的风格，默认使用单引号，将双引号作为备选（这次我还是用 ruff）

In [2]:
"I perfer single quotes"

'I perfer single quotes'

如果必须使用 black, 那么请使用 `black -S` 选项，保持引号原封不动

---

### 使用 None 表示默认值

如果可选的参数是一个可变类型，那么使用 None 是唯一合理的默认值

大部分情况下，使用 None 作为默认值更好

如果想把 plural 参数的默认值设置为 None, 则函数的签名要改成下面这样：

In [3]:
from typing import Optional

def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
    return f"{count}, {singular}, {plural}"
    

- Optional[str] = None 表示可选参数的默认值是 None
- 必须显式提供默认值，即 `=None`

## 注解中可用的类型

- `typing.Any`
- `简单的类型和类`
- `typing.Optional` 和 `typing.Union`
- 泛化容器，包括元组和映射
- 抽象基类
- 泛化可迭代对象
- 参数化泛型和 `TypeVar`
- `typing.Protocols` -- 静态鸭子类型的关键
- `typing.Callable`
- `typing.NoReturn`

### Any 类型

Any 类型是渐进式类型系统的基础，是动态类型

比如：
```python
def double(x):
    return x * 2
```

在类型检查工具来看，假定的类型信息如下：

```python
def double(x: Any) -> Any:
    return x * 2
```

渐进式类型系统中有一种关系：**相容**。

满足子类型关系必定是相容的，不过对于 Any 还有特殊的规定。

<span style="color: green">相容规则如下 </span>

1. 对 T1 及其子类型 T2，T2 与 T1 相容
2. 任何类型都与 Any 相容：声明为 Any 类型的参数接受任何类型的对象
3. Any 与任何类型都相容：始终可以把 Any 类型的对象传给预期其他类型的参数

诸如 int, float, str 和 bytes 这样的简单的类型可以直接在类型提示中使用。

标准库，外部包中的具体类，以及用户自定义的具体类，也可以在类型提示中使用


### Optional 类型和 Union 类型

`Optional[str]` 结构其实是 `Union[str, None]`的简写形式，表示可以是 str 或 None

> <mark>Python3.10 为 Optional 和 Union 提供的句法更好</mark>

`Union[str, bytes]` 可以写成 `str | bytes`，这种写法输入的内容更少，也不用从 `typing` 中导入 `Optional` 或 `Union`.

```python
plural: Optional[str] = None    # 旧句法
plural: str | None = None       # 新句法
```

<span style="color: red"> `|` 运算符还可以用于构建 `isinstance` 和 `issubclass` 的第二个参数，例如 `isinstance(x, int|str)`。 详见 PEP 604 </span>

In [ ]:
from typing import Union  # noqa: E402

def parse_token(token: str) -> Union[str, float]:
    try:
        return float(token)
    except ValueError:
        return token

`Union[]` 至少需要两种类型。嵌套的 Union 类型与扁平的 Union 类型效果相同。因此，下面的两种类型提示的效果是相同的

`Union[A, B, Union[C, D, E]]`

`Union[A, B, C, D, E]`

---

<mark> Union 所含的类型之间不应相容 </mark>


例如：`Union[int, float]` 就画蛇添足了，因为 int 和 float 相容。仅使用 float 注解的参数也接受 int 值

---

### 泛化容器

大多数 Python 的容器都是异构的。例如，在一个 list 中可以混合存放不同的类型。

然而，实际中这么做没有太多的意义。存入容器中的对象往往需要进一步处理

In [1]:
def tokenize(text: str) -> list[str]:
    return text.upper.split()

3.9 版本及以上，类型提示的意思是 `tokenize` 函数返回一个 list，且各项均为 `str` 类型

`stuff: list` 和 `stuff: list[Any]` 两者的意思相同，都表示 `stuff` 是一个列表，且列表中的项可以是任何类型的对象

---

如果想注解带有多个字段的元组，或者代码中多次用到的特定类型的元组，强烈建议使用 `typing.NamedTuple`

In [ ]:
from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float


def geohash(lat_lon: Coordinate) -> str:
    print(f"{lat_lon.lat}, {lat_lon.lon}")
    return "geohash"

---

### 泛化映射

泛化映射类型使用 `MappingType[KeyType, ValueType]` 形式注解.

在 Python3.9 及以上版本，内置类型 dict 和 collections 和 collections.abc 中的映射类型都可以这样注解

更早的版本必须使用 `typing.Dict` 和 `typing` 模块中的其他映射类型

<span style="color:red"> 函数的参数应接受那些**抽象类型**，或者Python3.9 之前的版本中的 typing 模块中对应的类型， 而不是具体类型 </span>，这样对调用方来说更加灵活

```python
from collections.abc import Mapping

def name2hex(name: str, color_map: Mapping[str, int]) -> str:

```

注解的类型是 `abc.Mapping`，因此调用方可以提供 `dict`, `defaultdict` 和 `ChainMap` 的实例，`UserDict` 子类的实例，或者 `Mapping` 的任何子类

相比之下，再看下面的签名

```python
def name2hex(name: str, color_map: dict[str, int]) -> str:
```

这里，`color_map` 必须是 dict 或者其子类型，例如 defaultDict 或 OrderedDict, 但是需要注意的是，无法使用 `collections.UserDict` 子类通过类型检查


<span style="color:green"> 一般来说在参数的类型提示中最好使用 `abc.Mapping` 或者 `abc.MutableMapping` </span>, 不要使用 `dict` （也不要在遗留代码中使用 `typing.Dict`

如果 name2hex 不改变传入的 color_map, 那么最准确的类型提示是 `abc.Mapping`

- 接收时要大方，所以入参最好足够抽象，方便接收更多的类型，
- 发送时要保守，因此函数的返回值始终应该是一个具体对象

> 泛化版 list，可用于注解返回值类型。
> 
> 如果想注解参数，推荐使用**抽象容器类型**，例如 `Sequence` 或者 `Iterable`

---

### `Iterable`

`typing.List` 推荐使用 `Sequence` 和 `Iterable` 注解函数的参数

```python
from collections.abc import Iterable

FromTo = tuple[str, str]
def zip_replace(text: str, change: Iterable[FromTo]) -> str:
    for from_, to in changes:
        text = text.replace(from_, to)

    return text
```

- `FromTo` 是类型别名，把 `tuple[str, str]` 赋给了 `FromTo`，这样 `zip_replace` 函数签名的可读性更好
- `changes` 的类型为 `Iterable[FromTo]`, 这与 `Iterable[tuple[str, str]]`的效果一样, 但是签名更短，可读性更高


> #### Python3.10 显示使用 TypeAlias

TypeAlias 让创建的类型别名的赋值操作更加显眼，也让类型检查更容易


```python
from typing import TypeAlias

FromTo: TypeAlias = tuple[str, str]
```


---

### 参数化泛型和 TypeVar

参数化泛型是一种泛型，写作 `list[T]`, 其中 T 是类型变量，每次使用时会绑定具体的类型

---

### `Callable`

`collections.abc` 模块提供的 `Callable` 类型用于注解返回的可调用对象

Callable 类型可以像下面这样参数化

`Callable[[ParamType1, ParamType2], ReturnType]`

参数列表，即 `[ParamType1, ParamType2]` 可以包含零或多个类型